In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Split data with knn on lat and long


In [2]:
# pd.read_csv('kaggle/input/foursquare-location-matching/pairs.csv')

In [3]:
# pd.read_csv('kaggle/input/foursquare-location-matching/test.csv')

In [4]:
pd.read_csv('kaggle/input/foursquare-location-matching/sample_submission.csv')

,id,matches
0,E_00001118ad0191,E_00001118ad0191
1,E_000020eb6fed40,E_000020eb6fed40
2,E_00002f98667edf,E_00002f98667edf
3,E_001b6bad66eb98,E_001b6bad66eb98 E_0283d9f61e569d
4,E_0283d9f61e569d,E_0283d9f61e569d E_001b6bad66eb98


In [5]:
df = pd.read_csv('kaggle/input/foursquare-location-matching/train.csv')

In [6]:
df.country.unique()

array(['BE', 'BR', 'TH', 'TR', 'ES', 'PH', 'KW', 'ID', 'US', 'DE', 'SA',
       'JP', 'MY', 'IT', 'FR', 'PT', 'EG', 'MX', 'UA', 'RU', 'KR', 'MV',
       'ZA', 'CL', 'HU', 'CA', 'SG', 'FI', 'RO', 'IR', 'SK', 'MK', 'CH',
       'GB', 'PL', 'CZ', 'AT', 'CN', 'RS', 'AR', 'KH', 'IN', 'TW', 'LV',
       'PY', 'DO', 'AE', 'VN', 'VE', 'PE', 'BY', 'IE', 'XX', 'MW', 'NL',
       'AU', 'DK', 'KZ', 'CR', 'PR', 'BG', 'GR', 'HK', 'GE', 'AG', 'CO',
       'MU', 'ME', 'MQ', 'AM', 'IL', 'NZ', 'MT', 'KE', 'PF', 'PA', 'SE',
       'BT', 'SI', 'CW', 'CY', 'AL', 'LB', 'NO', 'EE', 'BN', 'BH', 'LU',
       'FJ', 'MD', 'LT', 'TT', 'JO', 'LA', 'QA', 'TN', 'HR', 'AZ', 'RW',
       'SV', 'GH', 'EC', 'PK', 'UY', 'SR', 'JM', 'MA', 'ZM', 'AW', 'MO',
       'BS', 'LK', 'SY', 'UZ', 'NI', 'IS', 'TM', 'GU', 'GT', 'RE', 'BO',
       'IM', 'CU', 'BB', 'AO', 'NP', 'OM', 'NG', 'HN', 'KY', 'UG', 'MN',
       'BD', 'MM', 'ET', 'MC', 'KG', 'AX', 'MG', 'SD', 'KP', 'DZ', 'BL',
       'WS', 'HT', 'MZ', 'TC', 'YE', 'BA', 'CV', 'A

In [30]:
df.count()

id                   1138812
name                 1138811
latitude             1138812
longitude            1138812
address               742191
city                  839623
state                 718226
zip                   543386
country              1138801
url                   267724
phone                 342855
categories           1040505
point_of_interest    1138812
dtype: int64

In [27]:
df.describe()

,latitude,longitude
count,1.138812e+06,1.138812e+06
mean,2.687460e+01,2.070497e+01
std,2.314474e+01,8.267784e+01
min,-8.492774e+01,-1.751602e+02
25%,1.276506e+01,-7.143512e+01
50%,3.570244e+01,2.893202e+01
75%,4.167972e+01,1.022178e+02
max,8.505113e+01,1.784404e+02


In [14]:
values_count = df.point_of_interest.apply(lambda x: len(x))

In [11]:
df.groupby('point_of_interest').agg({'id':lambda x: ",".join(x)})

,id
point_of_interest,
P_00000458b5ff8e,E_b4a705f3f2e85e
P_00001c309a5e0a,"E_3b8a876a004093,E_7b12ba037d7af8,E_a66af82051..."
P_000047d288cb51,E_9822da0f40eaa3
P_0000561fe92bed,"E_2f12c350e37a69,E_e14714f4aa6461"
P_0000bccb92573c,"E_408896b60c7893,E_822bc2df39ec06"
...,...
P_ffff287983c417,"E_47d063fddd9115,E_ca42d07c618f1c"
P_ffff42b86df943,E_4d52ca0489f774
P_ffff47d56844f9,E_c32f2d8e15a529


In [ ]:
from sklearn.neighbors import KNeighborsRegressor
